In [ ]:
%matplotlib inline
import math
import pandas as pd
from pandas.plotting import table
import matplotlib.pyplot as plt
from IPython.display import display, Markdown, HTML

In [ ]:
#POOLSFILE = ['pools.csv', 'anotherpath/pools.csv']
POOLSFILE = ['/tmp/zfssa_explorer_XXX.XXX.XXX.XXX_XXXXXX_XXXXXX/pools.csv']

Just a function to change sizes to human readable for return values

In [ ]:
suffixes = ['B', 'KB', 'MB', 'GB', 'TB', 'PB']
def human_size(nbytes):
    i = 0
    while nbytes >= 1024 and i < len(suffixes)-1:
        nbytes /= 1024.
        i += 1
    f = ('%.2f' % nbytes).rstrip('0').rstrip('.')
    return '%s %s' % (f, suffixes[i])

In [ ]:
#dataraw = pd.read_csv(POOLSFILE, header=1, sep=";")
dataraw = ""
if len(POOLSFILE) == 1:
    dataraw = pd.read_csv(str(POOLSFILE).strip("[']"), header=1, sep=";")
else:
    filelist = []
    for file in POOLSFILE:
        x = 0
        dfname = "df{}".format(x)
        dfname = pd.read_csv(file, header=1, sep=";")
        filelist.append(dfname)
        x + 1
    dataraw = pd.concat(filelist)

Show available columns

In [ ]:
dataraw.columns

In [ ]:
# drop rows where the status is exported to avoid some duplicated values 
dataraw = dataraw[dataraw.status != 'exported']
countpools = dataraw[['asn']]
display(Markdown("### Pools: {}".format(len(countpools))))

check profiles

In [ ]:
profiles = dataraw[['profile', 'name', 'owner', 'status']].set_index('name')
profiles

check if more than one profile is in the same node

In [ ]:
uniqref = profiles[profiles['status'] != 'exported']
uniq = uniqref.groupby('name')['profile']
if uniq.all().nunique() != len(uniqref):
    display(HTML('<strong><span style="background-color: #FFFF00">'
                 'Some pools with the same profile are in the same controller (Not Recommended).'
                 '</span></strong>'))
else:
    display(HTML('<strong>Pools per profile distributed normally</strong>'))
uniq.all()

### Get usage distribution

legend:

    usage_total = total pool capacity
    usage_usage_total = total space used
    usage_usage_data = space used (not considering reservation)
    usage_used = real space used
    usage_available = space available - reservation, etc
    usage_free = space free for assigment
    usage_usage_snapshots = space used for snapshots

In [ ]:
poolsimported = dataraw.set_index('name')
poolsimported = poolsimported[profiles['status'] != 'exported']
usage = poolsimported[['usage_available', 'usage_used', 'usage_usage_data', 'usage_free',
                       'usage_usage_snapshots', 'usage_usage_total', 'usage_total']]
usage = usage.astype(float)
plt.figure(figsize=(30, 8))
# plot bar
ax1 = plt.subplot(121)
ax1 = usage.astype(float).plot(kind='bar', legend=True, ax=ax1, fontsize=12, grid=True)
ax1.set_ylabel('Space(TB)')
ax1.set_xlabel('pool')
plt.show()
usage.applymap(human_size)

### Get compression info

In [ ]:
compress = dataraw[['name', 'usage_compression']]
compress.set_index('name')